In [2]:
# version 2018 is quite easy
# download & extract
url = 'https://anaconda.org/rdkit/rdkit/2018.09.1.0/download/linux-64/rdkit-2018.09.1.0-py36h71b666b_1.tar.bz2'
!curl -L $url | tar xj lib
# move to python packages directory
!mv lib/python3.6/site-packages/rdkit /usr/local/lib/python3.6/dist-packages/
x86 = '/usr/lib/x86_64-linux-gnu'
!mv lib/*.so.* $x86/
# rdkit need libboost_python3.so.1.65.1
!ln -s $x86/libboost_python3-py36.so.1.65.1 $x86/libboost_python3.so.1.65.1

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3749    0  3749    0     0  19731      0 --:--:-- --:--:-- --:--:-- 19731
100 20.2M  100 20.2M    0     0  5208k      0  0:00:03  0:00:03 --:--:-- 5642k
mv: cannot move 'lib/python3.6/site-packages/rdkit' to '/usr/local/lib/python3.6/dist-packages/rdkit': Directory not empty
ln: failed to create symbolic link '/usr/lib/x86_64-linux-gnu/libboost_python3.so.1.65.1': File exists


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
ls

drive/  lib/  sample_data/


In [5]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
cd drug_discovery/

/content/drive/.shortcut-targets-by-id/19b_O5F-YXehQUaR7OjSc5qFpWmOOs-7k/drug_discovery


In [ ]:
ls

AA.csv  drug_discovery1.ipynb


In [173]:
import pandas as pd
data = pd.read_csv('/content/drive/My Drive/drug_discovery/AA.csv')
data

,zinc_id,smiles
0,ZINC000000008151,C[C@H]1[C@@H](O)[C@H](CO)O[C@@H](O)[C@@H]1N
1,ZINC000000008153,CC[C@@H]1[C@@H](N)[C@@H](O)O[C@@H](CO)[C@@H]1O
2,ZINC000000008155,CC1(C)[C@@H](N)[C@@H](O)O[C@@H](CO)[C@@H]1O
3,ZINC000000018276,CS[C@@H]1CN[C@@H](CO)[C@H](O)[C@H]1O
4,ZINC000000018279,CS[C@@H]1[C@@H](O)CN[C@@H](CO)[C@@H]1O
...,...,...
427851,ZINC000242463989,O[Cl+3](O)(O)O
427852,ZINC000247713634,O1[SiH2][SiH2]O[SiH2][SiH2]1
427853,ZINC000252581626,O[Si](O)(O)F
427854,ZINC000685945533,Cn1nnnc1S(=O)(=O)F


In [174]:
data['smiles'].apply(len).max()

62

In [175]:
data['smiles'].apply(len).min()

4

In [176]:
from rdkit import Chem

In [177]:
SMILES_CHARS = [' ',
                  '#', '%', '(', ')', '+', '-', '.', '/',
                  '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                  '=', '@',
                  'A', 'B', 'C', 'F', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P',
                  'R', 'S', 'T', 'V', 'X', 'Z',
                  '[', '\\', ']',
                  'a', 'b', 'c', 'e', 'g', 'i', 'l', 'n', 'o', 'p', 'r', 's',
                  't', 'u']
smi2index = dict( (c,i) for i,c in enumerate( SMILES_CHARS ) )
index2smi = dict( (i,c) for i,c in enumerate( SMILES_CHARS ) )
def smiles_encoder( smiles, maxlen=62 ):
    smiles = Chem.MolToSmiles(Chem.MolFromSmiles( smiles ))
    X = np.zeros( ( maxlen, len( SMILES_CHARS ) ) )
    for i, c in enumerate( smiles ):
        X[i, smi2index[c]] = 1
    return X
def smiles_decoder( X ):
    smi = ''
    X = X.argmax( axis=-1 )
    for i in X:
        smi += index2smi[i]
    return smi

In [178]:
import numpy as np
x = []
for i in data['smiles'][:500]:
    #print(smiles_encoder(i))
    x.append(smiles_encoder(i))
arr = np.array(x)

In [179]:
arr.shape

(500, 62, 56)

In [180]:
arr[0].shape

(62, 56)

In [181]:
arr = arr.reshape(-1,62,56,1)

In [182]:
arr.shape

(500, 62, 56, 1)

In [183]:
# #Hyperparameters

# #Number of epochs for training
# NUM_EPOCHS = 35

# #Hidden channel dimension
# hidden_channels = 32

# #Latent space dimension
# latent_dim = 196

# #Convolutional layer kernel size
# kernel_size = 3

# #Stride is the number of pixels shifts over the input matrix  
# stride = 1

# #Padding size to add around the matrix.
# padding = 1

In [184]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.patches as mpatches 
%matplotlib inline
from scipy.stats import norm 
import keras
from keras import layers
from keras.models import Model 
from keras import metrics
from keras import backend as K 
K.clear_session()
np.random.seed(237)
import scipy.stats as st
import tensorflow.compat.v1.keras.backend as K 
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.optimizers import Adam 
import warnings 
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
import numpy as np
from scipy.stats import multivariate_normal

In [185]:
img_shape = (62, 56,1)
batch_size = 16
latent_dim = 2
input_img = keras.Input(shape=img_shape)
x = layers.Conv1D(32, 7,
                  padding='same',
                  activation='relu',
                  )(input_img)
x = layers.Conv2D(128, 3,
                  padding='same',
                  activation='relu',
                  strides=(2, 2))(x)
x = layers.Conv2D(128, 3,
                  padding='same',
                  activation='relu')(x)
x = layers.Conv2D(128, 3,
                  padding='same',
                  activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)
z_mu = layers.Dense(latent_dim)(x)
z_log_sigma = layers.Dense(latent_dim)(x)

In [186]:
def sampling(args):
  z_mu, z_log_sigma = args
  epsilon = K.random_normal(shape=(K.shape(z_mu)[0], latent_dim),
  mean=0., stddev=1.)
  return z_mu + K.exp(z_log_sigma) * epsilon
z = layers.Lambda(sampling)([z_mu, z_log_sigma])

In [187]:
decoder_input = layers.Input(K.int_shape(z)[1:])
x = layers.Dense(np.prod(shape_before_flattening[1:]),
                 activation='relu')(decoder_input)
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3,
                           padding='same',
                           activation='relu',
                           strides=(2, 2))(x)
x = layers.Conv2D(1, 3,
                  padding='same',
                  activation='sigmoid')(x)
decoder = Model(decoder_input, x)
z_decoded = decoder(z)

In [188]:
class CustomVariationalLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded) #Recin loss
        kl_loss = -5e-4 * K.mean(1 + z_log_sigma - K.square(z_mu) - K.exp(z_log_sigma), axis=-1) #KL loss
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        self.add_loss(loss, inputs=inputs)
        return x

y = CustomVariationalLayer()([input_img, z_decoded])

In [189]:
# VAE model statement
vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None)
vae.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 62, 56, 1)]  0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 62, 56, 32)   256         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 31, 28, 128)  36992       conv1d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 31, 28, 128)  147584      conv2d[0][0]                     
____________________________________________________________________________________________

In [190]:
vae.fit(x=arr, y=None,
        shuffle=False,
        epochs=120,
        batch_size=batch_size,
        )

Train on 500 samples
Epoch 1/120
500/500 [==============================] - 1s 1ms/sample - loss: 0.2486
Epoch 2/120
500/500 [==============================] - 0s 422us/sample - loss: 0.0448
Epoch 3/120
500/500 [==============================] - 0s 413us/sample - loss: 0.0388
Epoch 4/120
500/500 [==============================] - 0s 414us/sample - loss: 0.0363
Epoch 5/120
500/500 [==============================] - 0s 412us/sample - loss: 0.0351
Epoch 6/120
500/500 [==============================] - 0s 417us/sample - loss: 0.0347
Epoch 7/120
500/500 [==============================] - 0s 414us/sample - loss: 0.0343
Epoch 8/120
500/500 [==============================] - 0s 407us/sample - loss: 0.0339
Epoch 9/120
500/500 [==============================] - 0s 416us/sample - loss: 0.0338
Epoch 10/120
500/500 [==============================] - 0s 418us/sample - loss: 0.0337
Epoch 11/120
500/500 [==============================] - 0s 422us/sample - loss: 0.0337
Epoch 12/120
500/500 [===========

In [191]:
# sample_vector = np.array([[0,5]])
# result = decoder.predict(sample_vector)

In [192]:
# result.shape

In [193]:
# result = result[0].reshape(62,56)

In [194]:
# result.shape

In [196]:
# smiles_decoder(result)

In [214]:
import random
result = []
for i in random.sample(range(1, 10000000), 100):
  for j in random.sample(range(1, 10000000), 100):
    sample_vector = np.array([[i,j]])
    res = decoder.predict(sample_vector)
    res = res[0].reshape(62,56)
    result.append(smiles_decoder(res).replace(' ',''))
    # print(i,j)
set(result)

{'CC[C@@H]1O[C@@H]())C@@H](O)[C@@H](O)[C@@H1OO',
 'CC[C@@H]1O[C@@H]())C@H](O)[C@@H](O)[C@@H11O',
 'CC[C@@H]1O[C@@H]())C@H](O)[C@@H](O)[C@@H1OO',
 'CC[C@@H]1O[C@@H]())CC@H](O)[C@@H](O)[C@@H11O',
 'CC[C@@H]1O[C@@H](N))CC@H](O)[C@@H](O)[C@@H11O',
 'C[C@@H]1O[C@@H]())C@@H](O)[C@@H](O)[C@@H1OO',
 'C[C@@H]1O[C@@H](O))C@@H](O)[C@@H](O)[C@@H1OO',
 'OC[C@@H]1O[C@@H](O))C@@H](O)[@@H](O)[C@H]1O',
 'OC[C@@H]1O[C@@H](O))C@@H](O)[C@@H](O)[C@@H1O',
 'OC[C@@H]1O[C@@H](O))C@@H](O)[C@@H](O)[C@@H1OO',
 'OC[C@@H]1O[C@@H](O))C@@H](O)[C@@H](O)[C@HH1O',
 'OC[C@@H]1O[C@@H](O))C@@H](O)[C@@H](O)[C@H]1O',
 'OOCCH(1O@H(O)[H([[(@H]O@@H]O',
 'OOCCH(1O[@H(O)[@H([[(@H]O@@H]O',
 'OOCCH(1O[@H(O)[H([[(@H]O@@H]O',
 'OOCCH(1O[@H](O)[@H([[(@H]O@@H]O',
 'OOCCH(1O[@H](O)[@H([[@HHO@@H]O',
 'OOCCH(1O[@H](O)[@H([[@H]O@@H]O',
 'OOCH((O)()H((]@]',
 'OOCH((O)()H(]@]',
 'OOCH((O)()H]@]',
 'OOCH((O)()H]]',
 'OOCH((O)(H)H]]',
 'OOCH(1O(O)](H(@H](@@H]',
 'OOCH(1O(O)](H(@H](@H]',
 'OOCH(1O(O)](H(H](@H]',
 'OOCH(1O(O)]([(@H](@@H]',
 'OO

In [ ]:
# import rdkit
# rdkit.Chem.MolDrawing(smiles_decoder(result))

In [202]:
# result = list(map(lambda x: x.replace(' ',''),result))

In [215]:
# result

In [216]:
from rdkit.Chem import Draw
for i in result:
  try:
    m = Chem.MolFromSmiles(smiles_decoder(i))
    fig = Draw.MolToMPL(m, size=(200,200))
    print('possible')
  except:
    continue  

In [212]:
import pickle
pickle.dump(decoder, open('/content/drive/My Drive/decoder.pkl','wb'))

TypeError: ignored

In [ ]:
'/content/drive/My Drive/decoder.pkl'